In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import davies_bouldin_score,silhouette_score
import tensorflow as tf

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def initialization(X,k,state):
  random.seed(state)
  c = random.sample(range(0, len(X)), k,) #random  k points from the data
  centroids = []
  for i in c:
    centroids.append(np.array(X[i]))
  return centroids

In [ ]:
def calc_dist(X1, X2):
    return(sum((X1 - X2)**2))**0.5

In [ ]:
def findlabels(ic, X):
    labels = []
    for i in X:
        distance=[]
        for j in ic:
            distance.append(calc_dist(i, j))
        labels.append(np.argmin(distance))
    return labels

In [ ]:
def calc_centroids(clusters, X):
    new_centroids = []
    new_df = pd.concat([pd.DataFrame(X), pd.DataFrame(clusters, columns=['cluster'])],
                      axis=1)
    for c in set(new_df['cluster']):
        current_cluster = new_df[new_df['cluster'] == c][new_df.columns[:-1]]
        cluster_mean = current_cluster.mean(axis=0)
        new_centroids.append(np.array(cluster_mean))
    return new_centroids

In [ ]:
def kmeans(X,k,iterations,seed):
    centroids=initialization(X,k,seed) #initial centroids
    labels=[]
    for itr in range(iterations):
      labels = findlabels(centroids, X) #assigned cluster to each
      new_centroids = calc_centroids(labels,X)
      if np.array_equal(new_centroids,centroids):
        return labels
      centroids= new_centroids
    return labels

In [ ]:
with tf.device('/device:GPU:0'):
  sv=np.zeros((56,9))
  db=np.zeros((56,9))
  for i in range(1,57):
      print(i)
      df=pd.read_csv('/content/drive/My Drive/DM_assignment/'+str(i)+'.csv',header= None)
      X=np.array(df.iloc[:,0:-1])
      for j in range(2,11):
        labels = kmeans(X,j,300,0)
        sv[i-1,j-2]=silhouette_score(X,labels)
        db[i-1,j-2]=davies_bouldin_score(X,labels)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [ ]:
df1=pd.DataFrame(sv)
df2=pd.DataFrame(db)
df1.to_csv('/content/drive/My Drive/DM_assignment_/svkmeans.csv')
df2.to_csv('/content/drive/My Drive/DM_assignment_/dbkmeans.csv')
